<a href="https://colab.research.google.com/github/komazawa-deep-learning/komazawa-deep-learning.github.io/blob/master/2023notebooks/2023_0623stable_baselines3_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
date: 2023_0619
author: 浅川伸一
---

# stable-baselines3 を用いた強化学習のデモ

source from [https://araffin.github.io/post/sb3/](https://araffin.github.io/post/sb3/)

<!-- <center>
<div>
<video controls src="https://youtu.be/M1_qCqvW-u4" muted="false">
</video>
</center>
 -->

In [ ]:
from IPython import get_ipython
isColab =  'google.colab' in str(get_ipython())

# Google Colaboratory 上で実行するなら，必要なライブラリをインストールする
if isColab:
    !apt-get install ffmpeg freeglut3-dev xvfb  # For visualization
    !pip install "stable-baselines3[extra]>=2.0.0a4" --upgrade
    !pip install swig --upgrade
    !pip install 'gymnasium[box2d]' --upgrade

import gymnasium as gym
import numpy as np

### 0. 動画撮影の準備

フェイク画面のセットアップ，そうしないとレンダリングに失敗する.

MacOS 上での実行には，以下のコマンドライン操作が，一度だけ必要

```bash
mkdir /private/tmp/.X11-unix
sudo chown root /private/tmp/.X11-unix
sudo chmod 1777 /private/tmp/.X11-unix
```


In [ ]:
# フェイク画面のセットアップ，そうしないとレンダリングに失敗する
# MacOS 上での実行には，以下のコマンドライン操作が，一度だけ必要
#
#```bash
# mkdir /private/tmp/.X11-unix
# sudo chown root /private/tmp/.X11-unix
# sudo chmod 1777 /private/tmp/.X11-unix
#```
import os
os.system("Xvfb :1 -screen 0 1024x768x24 &")
os.environ['DISPLAY'] = ':1'

In [ ]:
import base64
from pathlib import Path
from IPython import display as ipythondisplay

def show_videos(video_path="", prefix=""):
    """
    保存してある動画を再生

    :param video_path: (str) 動画が格納されているフォルダのパス
    :param prefix: (str) この接頭辞のついた動画のみを表示する

    https://github.com/eleurent/highway-env より借用
    """
    html = []
    for mp4 in Path(video_path).glob("{}*.mp4".format(prefix)):
        video_b64 = base64.b64encode(mp4.read_bytes())
        html.append(
            """<video alt="{}" autoplay
                    loop controls style="height: 400px;">
                    <source src="data:video/mp4;base64,{}" type="video/mp4" />
                </video>""".format(
                mp4, video_b64.decode("ascii")
            )
        )
    ipythondisplay.display(ipythondisplay.HTML(data="<br>".join(html)))

from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(env_id,
                 model, video_length=500,
                 prefix="",
                 video_folder="videos/"):
    """エージェントの動作を動画にして保存

    :param env_id: (str)
    :param model: (RL model)
    :param video_length: (int)
    :param prefix: (str)
    :param video_folder: (str)
    """
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Start the video at step=0 and record 500 steps
    eval_env = VecVideoRecorder(
        eval_env,
        video_folder=video_folder,
        record_video_trigger=lambda step: step == 0,
        video_length=video_length,
        name_prefix=prefix,
    )

    obs = eval_env.reset()
    for _ in range(video_length):
        action, _ = model.predict(obs)
        obs, _, _, _ = eval_env.step(action)

    # Close the video recorder
    eval_env.close()

# 1. 強化学習モデルの定義と訓練の実施

stable baselines3 の公式ドキュメントによれば，`LunarLander` を実行するには，`bod2d` モジュールが必要になる。
[ローカル文書](file:///Users/_asakawa/study/2021DLR-RM_stable-baselines3.git/docs/_build/html/guide/examples.html#basic-usage-training-saving-loading)
```
LunarLander requires the python package box2d.
You can install it using `apt install swig` and then `pip install box2d box2d-kengz`
```

In [ ]:
# source form <https://araffin.github.io/post/sb3/>
from stable_baselines3 import PPO  # Proximal Policy Optimization
from stable_baselines3 import A2C  # Asynchronous Advantage Actor Critic (A3C)
from stable_baselines3 import DDPG # Deep Deterministic Policy Gradient
from stable_baselines3 import DQN  # Deep Q Network
from stable_baselines3 import SAC  # Soft Actor Critic
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback

# 途中経過 `checkpoint` を 5000 ステップ毎に保存
checkpoint_callback = CheckpointCallback(save_freq=5000,
                                         save_path="./logs/",
                                         name_prefix="rl_model")

# モデルを定期的に評価し，最良モデルを自動的に保存する
# エピソードの統計量を報告させるには，適切なモニターラッパが必要
game_id = 'Breakout'  # 'CartPole-v2', 'LunarLander-v2', 'Tennis'
game_id = 'Tennis'
game_id = 'Boxing'
game_id = 'Gopher'
game_id = 'CartPole-v1'
eval_env = Monitor(gym.make(game_id))
#eval_env = Monitor(gym.make("LunarLander-v2"))

# 評価のためには，決定論的な行動が必要
eval_callback = EvalCallback(eval_env,                        # モニター環境
                             best_model_save_path="./logs/",  # 最良モデルの保存場所
                             log_path="./logs/",              # ログの保存場所
                             eval_freq=2000,                  # 観察頻度
                             deterministic=True,              # 決定論的行動
                             render=False)                    # レンダリング

# 動作主 (エージェント) を訓練
model = PPO("MlpPolicy", game_id, verbose=0)

# 実際の学習
model.learn(total_timesteps=20000,
            callback=[checkpoint_callback, eval_callback])

# 環境の呼び出しとリセット
env = model.get_env()
obs = env.reset()

# モデルを呼び出して，確率論的な動作をさせる
action, _ = model.predict(obs, deterministic=False)


In [ ]:
env_id=game_id
prefix=game_id
record_video(env_id=env_id,
             model=model,
             video_length=500,
             prefix=prefix)

In [ ]:
show_videos(video_path="videos", prefix=prefix)